In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('bitcoinfinal+(4).csv',header=None)
df.columns = ['Month','Bitcoin_Price']

In [5]:
df = df.set_index('Month')
df.head()

,Bitcoin_Price
Month,
0,217.4
1,255.7
2,244.3
3,236.1
4,228.7


In [6]:
train_len = 29
train = df[:train_len]
test = df[train_len:]

### Naive Method

In [8]:
y_hat_naive = test.copy()
y_hat_naive['naive_forecast'] = train['Bitcoin_Price'][train_len-1]
y_hat_naive['naive_forecast']

Month
29    2191.8
30    2191.8
31    2191.8
Name: naive_forecast, dtype: float64

In [11]:
## rmse and mape
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_naive['naive_forecast']))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_naive['naive_forecast'])/test['Bitcoin_Price']*100),2)
rmse,mape

(1265.9, 27.12)

In [12]:
results = pd.DataFrame({
    'Method':['Naive Forecast'],
    'RMSE':rmse,
    'MAPE':mape
})
results

,Method,RMSE,MAPE
0,Naive Forecast,1265.9,27.12


### Simple Average Method

In [10]:
y_hat_sa = test.copy()
y_hat_sa['sa_forecast'] = train['Bitcoin_Price'].mean()
y_hat_sa['sa_forecast']

Month
29    595.751724
30    595.751724
31    595.751724
Name: sa_forecast, dtype: float64

In [13]:
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_sa['sa_forecast']))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_sa['sa_forecast'])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Simple Average Method',rmse,mape]

In [14]:
results

,Method,RMSE,MAPE
0,Naive Forecast,1265.90,27.12
1,Simple Average Method,2703.73,80.19


### Moving Average

In [17]:
y_hat_ma_12 = df.copy()
y_hat_ma_12['ma_forecast_12'] = df['Bitcoin_Price'].rolling(12).mean()
y_hat_ma_12['ma_forecast_12'][train_len:] = y_hat_ma_12['ma_forecast_12'][train_len-1]
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_ma_12['ma_forecast_12'][train_len:]))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_ma_12['ma_forecast_12'][train_len:])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Moving Average Method_12',rmse,mape]

In [18]:
results

,Method,RMSE,MAPE
0,Naive Forecast,1265.90,27.12
1,Simple Average Method,2703.73,80.19
2,Moving Average Method_12,2338.94,67.42


In [19]:
y_hat_ma_6 = df.copy()
y_hat_ma_6['ma_forecast_6'] = df['Bitcoin_Price'].rolling(6).mean()
y_hat_ma_6['ma_forecast_6'][train_len:] = y_hat_ma_6['ma_forecast_6'][train_len-1]
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_ma_6['ma_forecast_6'][train_len:]))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_ma_6['ma_forecast_6'][train_len:])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Moving Average Method_6',rmse,mape]

In [20]:
y_hat_ma_3 = df.copy()
y_hat_ma_3['ma_forecast_3'] = df['Bitcoin_Price'].rolling(3).mean()
y_hat_ma_3['ma_forecast_3'][train_len:] = y_hat_ma_3['ma_forecast_3'][train_len-1]
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_ma_3['ma_forecast_3'][train_len:]))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_ma_3['ma_forecast_3'][train_len:])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Moving Average Method_3',rmse,mape]
results

,Method,RMSE,MAPE
0,Naive Forecast,1265.90,27.12
1,Simple Average Method,2703.73,80.19
2,Moving Average Method_12,2338.94,67.42
3,Moving Average Method_6,2035.79,56.61
4,Moving Average Method_3,1794.92,47.81


### Simple Exponential Smoothing

In [21]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
model = SimpleExpSmoothing(train['Bitcoin_Price'])
model_fit = model.fit(optimized=True)
print(model_fit.params)
y_hat_ses = test.copy()
y_hat_ses['ses_forecast'] = model_fit.forecast(len(test))
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_ses['ses_forecast']))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_ses['ses_forecast'])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Simple Exponential Smoothing',rmse,mape]
results

{'smoothing_level': 0.995, 'smoothing_trend': nan, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': 217.4, 'initial_trend': nan, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}


,Method,RMSE,MAPE
0,Naive Forecast,1265.90,27.12
1,Simple Average Method,2703.73,80.19
2,Moving Average Method_12,2338.94,67.42
3,Moving Average Method_6,2035.79,56.61
4,Moving Average Method_3,1794.92,47.81
5,Simple Exponential Smoothing,1268.86,27.25


### Holt Exponential Smoothing

In [22]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
model = ExponentialSmoothing(np.asarray(train['Bitcoin_Price']),seasonal_periods=12,trend='additive',seasonal=None)
model_fit = model.fit(optimized=True)
print(model_fit.params)

{'smoothing_level': 0.995, 'smoothing_trend': 0.4264285714285714, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': 222.41999999999987, 'initial_trend': 5.2600000000000176, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}


In [23]:
y_hat_hes=test.copy()
y_hat_hes['hes_forecast'] = model_fit.forecast(len(test))
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_hes['hes_forecast']))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_hes['hes_forecast'])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Holts Exponential Smoothing',rmse,mape]
results

,Method,RMSE,MAPE
0,Naive Forecast,1265.90,27.12
1,Simple Average Method,2703.73,80.19
2,Moving Average Method_12,2338.94,67.42
3,Moving Average Method_6,2035.79,56.61
4,Moving Average Method_3,1794.92,47.81
5,Simple Exponential Smoothing,1268.86,27.25
6,Holts Exponential Smoothing,497.91,10.99


### Holt Winters Additive Method

In [25]:
y_hat_hwa = test.copy()
model = ExponentialSmoothing(np.asarray(train['Bitcoin_Price']),seasonal_periods=3,trend='add',seasonal='add')
model_fit = model.fit(optimized=True)
y_hat_hwa['hwa_forecast'] = model_fit.forecast(len(test))
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_hwa['hwa_forecast']))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_hwa['hwa_forecast'])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Holt Winters Additive Method',rmse,mape]

### Holt Winters Multiplicative Method

In [26]:
y_hat_hwm = test.copy()
model = ExponentialSmoothing(np.asarray(train['Bitcoin_Price']),seasonal_periods=3,trend='add',seasonal='mul')
model_fit = model.fit(optimized=True)
y_hat_hwm['hwm_forecast'] = model_fit.forecast(len(test))
rmse = np.sqrt(np.mean(mean_squared_error(test['Bitcoin_Price'],y_hat_hwm['hwm_forecast']))).round(2)
mape = np.round(np.mean(np.abs(test['Bitcoin_Price']-y_hat_hwm['hwm_forecast'])/test['Bitcoin_Price']*100),2)
results.loc[len(results)] = ['Holt Winters Multiplicative Method',rmse,mape]
results

,Method,RMSE,MAPE
0,Naive Forecast,1265.90,27.12
1,Simple Average Method,2703.73,80.19
2,Moving Average Method_12,2338.94,67.42
3,Moving Average Method_6,2035.79,56.61
4,Moving Average Method_3,1794.92,47.81
5,Simple Exponential Smoothing,1268.86,27.25
6,Holts Exponential Smoothing,497.91,10.99
7,Holt Winters Additive Method,500.38,10.28
8,Holt Winters Multiplicative Method,683.59,15.14
